In [1]:
import tensorflow as tf
from tensorflowonspark import TFCluster
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from tensorflowonspark import TFCluster
import pyspark

2022-02-22 19:27:04.819592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-22 19:27:04.819648: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def main_fun(args, ctx):
    import numpy as np
    import tensorflow as tf
    from tensorflowonspark import compat, TFNode

    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

    def build_and_compile_cnn_model():
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
        model.compile(
            loss=tf.keras.losses.sparse_categorical_crossentropy,
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
            metrics=['accuracy'])
        return model

    # single node
    # single_worker_model = build_and_compile_cnn_model()
    # single_worker_model.fit(x=train_datasets, epochs=3)

    tf_feed = TFNode.DataFeed(ctx.mgr, False)

    def rdd_generator():
        while not tf_feed.should_stop():
            batch = tf_feed.next_batch(1)
            if len(batch) > 0:
                example = batch[0]
                image = np.array(example[0]).astype(np.float32) / 255.0
                image = np.reshape(image, (28, 28, 1))
                label = np.array(example[1]).astype(np.float32)
                label = np.reshape(label, (1,))
                yield (image, label)
            else:
                return

    ds = tf.data.Dataset.from_generator(rdd_generator, (tf.float32, tf.float32),
                                        (tf.TensorShape([28, 28, 1]), tf.TensorShape([1])))
    ds = ds.batch(args['batch_size'])

    # this fails
    # callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=args.model_dir)]
    tf.io.gfile.makedirs(args['model_dir'])
    filepath = args['model_dir'] + "/weights-{epoch:04d}"
    callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=filepath, verbose=1, save_weights_only=True)]

    with strategy.scope():
        multi_worker_model = build_and_compile_cnn_model()

    # Note: MultiWorkerMirroredStrategy (CollectiveAllReduceStrategy) is synchronous,
    # so we need to ensure that all workers complete training before any of them run out of data from the RDD.
    # And given that Spark RDD partitions (and partition sizes) can be non-evenly divisible by num_workers,
    # we'll just stop training at 90% of the total expected number of steps.
    steps_per_epoch = 60000 / args['batch_size']
    steps_per_epoch_per_worker = steps_per_epoch / ctx.num_workers
    max_steps_per_worker = steps_per_epoch_per_worker * 0.9

    multi_worker_model.fit(x=ds, epochs=args['epochs'], steps_per_epoch=max_steps_per_worker, callbacks=callbacks)

    compat.export_saved_model(multi_worker_model, args['export_dir'], ctx.job_name == 'chief')

    # terminating feed tells spark to skip processing further partitions
    tf_feed.terminate()


In [3]:
sc = SparkContext(conf=SparkConf().setMaster('spark://master:7077').setAppName("mnist_keras"))

22/02/22 19:27:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
sc

<SparkContext master=spark://master:7077 appName=mnist_keras>

In [5]:
executors = sc._conf.get("spark.executor.instances")
num_executors = int(executors) if executors is not None else 1

In [6]:
args = {
    'batch_size': 16,
    'cluster_size': num_executors,
    'epochs': 3,
    'images_labels': 'hdfs://hdfs:9000/user/mnist/csv/train',
    'model_dir': 'hdfs://hdfs:9000/user/mnist/model',
    'export_dir': 'hdfs://hdfs:9000/user/mnist/export'
}

In [7]:
def parse(ln):
    vec = [int(x) for x in ln.split(',')]
    return (vec[1:], vec[0])

images_labels = sc.textFile(args['images_labels']).map(parse)

In [8]:
cluster = TFCluster.run(sc, 
                        main_fun, 
                        args, 
                        args['cluster_size'],
                        num_ps=0,
                        input_mode=TFCluster.InputMode.SPARK,
                        master_node='chief')

2022-02-22 19:27:26,687 INFO (MainThread-75) Reserving TFSparkNodes 
2022-02-22 19:27:26,691 INFO (MainThread-75) cluster_template: {'chief': [0]}
2022-02-22 19:27:26,696 INFO (MainThread-75) Reservation server binding to port 0
2022-02-22 19:27:26,698 INFO (MainThread-75) listening for reservations at ('172.20.0.5', 40053)
2022-02-22 19:27:26,706 INFO (MainThread-75) Starting TensorFlow on executors
2022-02-22 19:27:26,735 INFO (MainThread-75) Waiting for TFSparkNodes to start
2022-02-22 19:27:26,739 INFO (MainThread-75) waiting for 1 reservations
2022-02-22 19:27:27,740 INFO (MainThread-75) waiting for 1 reservations+ 1) / 1]
2022-02-22 19:27:28,745 INFO (MainThread-75) waiting for 1 reservations
2022-02-22 19:27:29,748 INFO (MainThread-75) waiting for 1 reservations
2022-02-22 19:27:30,751 INFO (MainThread-75) waiting for 1 reservations
2022-02-22 19:27:31,756 INFO (MainThread-75) all reservations completed
2022-02-22 19:27:31,758 INFO (MainThread-75) All TFSparkNodes started
2022-0

In [ ]:
cluster.train(images_labels, args['epochs'])
cluster.shutdown()

2022-02-22 19:27:37,330 INFO (MainThread-75) Feeding training data


In [ ]:
cluster.train()